In [24]:
class abstract_graph:
    
    def __init__(self,_edges):
        self.edges=_edges
        self.nodes={u for u,v in self.edges} | {v for u,v in self.edges}
    
    def adjacency_list(self):
        pass
    
class simple_graph(abstract_graph):
    
    def __init__(self,_edges):
        self.edges=_edges
        self.nodes={u for u,v in self.edges} | {v for u,v in self.edges}
        
    def adjacency_matrix(self):
        # completar
        n=len(self.nodes)
        mat=np.zeros((n,n))
        return mat
    
    
    def adjacency_list(self):
        adjacent=lambda n : {v for u,v in self.edges if u==n } | {u for u,v in self.edges if v==n}
        return {v:adjacent(v) for v in self.nodes}
        

In [25]:
import numpy as np
random_graph = lambda n,p : {(u,v) for u in np.arange(n) for v 
                    in np.arange(n) if np.random.rand()>p and u!=v}
# cantidad de vértices
n=10
# 1- probabilidad de crear aristas
p=0.1

E=random_graph(n,p)
G=simple_graph(E)
print G.adjacency_list()

set([(5, 9), (4, 7), (1, 3), (9, 1), (4, 8), (3, 0), (2, 8), (8, 3), (8, 0), (5, 4), (0, 7), (8, 9), (5, 6), (6, 2), (1, 6), (9, 4), (5, 1), (2, 5), (8, 5), (0, 3), (5, 8), (4, 0), (1, 2), (7, 4), (9, 0), (9, 5), (6, 7), (8, 1), (7, 6), (6, 3), (1, 5), (7, 2), (3, 6), (0, 4), (5, 7), (5, 3), (4, 1), (9, 8), (9, 7), (2, 7), (3, 2), (2, 6), (5, 0), (7, 1), (4, 5), (7, 9), (8, 6), (6, 0), (1, 4), (3, 9), (2, 3), (2, 1), (6, 8), (9, 3), (4, 2), (6, 4), (0, 8), (9, 6), (3, 5), (0, 1), (6, 9), (7, 3), (4, 6), (9, 2), (6, 1), (3, 1), (8, 2), (0, 2), (3, 8), (2, 0), (1, 8), (7, 5), (4, 3), (0, 9), (0, 5), (8, 4), (5, 2), (2, 4), (1, 9)])
{0: set([1, 2, 3, 4, 5, 6, 7, 8, 9]), 1: set([0, 2, 3, 4, 5, 6, 7, 8, 9]), 2: set([0, 1, 3, 4, 5, 6, 7, 8, 9]), 3: set([0, 1, 2, 4, 5, 6, 7, 8, 9]), 4: set([0, 1, 2, 3, 5, 6, 7, 8, 9]), 5: set([0, 1, 2, 3, 4, 6, 7, 8, 9]), 6: set([0, 1, 2, 3, 4, 5, 7, 8, 9]), 7: set([0, 1, 2, 3, 4, 5, 6, 9]), 8: set([0, 1, 2, 3, 4, 5, 6, 9]), 9: set([0, 1, 2, 3, 4, 5, 6, 7, 8]

In [27]:
# https://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/

def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    for next in graph[start] - visited:
        dfs(graph, next, visited)
    return visited

def bfs(graph, start):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.add(vertex)
            queue.extend(graph[vertex] - visited)
    return visited

print(bfs(G.adjacency_list(),1))
print(dfs(G.adjacency_list(),1))

set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
